In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#model_file = '/content/drive/MyDrive/GNCIPL_AI_ML/best_stroke_model_augmented.pkl'
model_file = '/content/drive/Shared with me/Stroke_GenAI_Project/best_stroke_model_augmented.pkl'

In [ ]:
model_file

'/content/drive/Shared with me/Stroke_GenAI_Project/best_stroke_model_augmented.pkl'

In [ ]:
!pip install pyngrok

In [ ]:
!pip install streamlit

In [ ]:
import os
from threading import Thread
from pyngrok import ngrok
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import os


In [ ]:
MODEL_PATH = model_file

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib

# -------------------------------------------------
# App Config
# -------------------------------------------------
st.set_page_config(
    page_title="Stroke Risk Prediction",
    page_icon="🧠",
    layout="centered"
)

MODEL_PATH = "best_stroke_model_augmented.pkl"

# FIXED feature order (exactly as your trained model)
FEATURE_ORDER = [
    "age",
    "hypertension",
    "heart_disease",
    "avg_glucose_level",
    "bmi",
    "ever_married",
    "Residence_type",
    "gender_male",
    "gender_female",
    "gender_other",
    "work_type_Private",
    "work_type_Self-employed",
    "work_type_Govt_job",
    "work_type_children",
    "work_type_Never_worked",
    "smoking_status_never smoked",
    "smoking_status_formerly smoked",
    "smoking_status_smokes",
    "smoking_status_Unknown"
]

# -------------------------------------------------
# Load model
# -------------------------------------------------
@st.cache_resource
def load_model():
    return joblib.load(MODEL_PATH)
model = load_model()


# -------------------------------------------------
# UI Header
# -------------------------------------------------
st.markdown(
    """
    <h2 style='text-align:center;'>🧠 Stroke Risk Prediction System</h2>
    <p style='text-align:center;color:gray;'>
    AI-powered prediction using clinical & lifestyle data
    </p>
    """,
    unsafe_allow_html=True
)

# -------------------------------------------------
# Sidebar Inputs
# -------------------------------------------------
st.sidebar.header("🧾 Patient Details")

age = st.sidebar.slider("Age", 0, 120, 45)
avg_glucose = st.sidebar.number_input(
    "Average Glucose Level (mg/dL)",
    min_value=40.0,
    max_value=400.0,
    value=100.0,
    step=1.0
)
bmi = st.sidebar.number_input(
    "BMI",
    min_value=10.0,
    max_value=60.0,
    value=25.0,
    step=0.1
)


hypertension = st.sidebar.selectbox("Hypertension (0 = No, 1 = Yes)", [0, 1])
heart_disease = st.sidebar.selectbox("Heart Disease (0 = No, 1 = Yes)", [0, 1])

ever_married = st.sidebar.selectbox("Ever Married (0 = No, 1 = Yes)", [0, 1])
Residence_type = st.sidebar.selectbox("Residence Type (0 = Rural, 1 = Urban)", [0, 1])

gender = st.sidebar.selectbox("Gender", ["male", "female", "other"])
work_type = st.sidebar.selectbox(
    "Work Type",
    ["Private", "Self-employed", "Govt_job", "children", "Never_worked"]
)
smoking_status = st.sidebar.selectbox(
    "Smoking Status",
    ["never smoked", "formerly smoked", "smokes", "Unknown"]
)

# -------------------------------------------------
# Build input row (ALL ZERO BY DEFAULT)
# -------------------------------------------------
def build_input():
    row = pd.Series(0, index=FEATURE_ORDER, dtype=float)

    row["age"] = age
    row["avg_glucose_level"] = avg_glucose
    row["bmi"] = bmi
    row["hypertension"] = hypertension
    row["heart_disease"] = heart_disease
    row["ever_married"] = ever_married
    row["Residence_type"] = Residence_type

    row[f"gender_{gender}"] = 1
    row[f"work_type_{work_type}"] = 1
    row[f"smoking_status_{smoking_status}"] = 1

    return pd.DataFrame([row])


# -------------------------------------------------
# Prediction
# -------------------------------------------------
st.markdown("---")

if st.button("🔍 Predict Stroke Risk", width="stretch"):

    X = build_input()  # Build the input row as before
    pred = model.predict(X)[0]
    proba = model.predict_proba(X)[0][1]  # probability of stroke

    # ----------------------------
    # Risk Level Classification
    # ----------------------------
    if proba < 0.5:
        risk_level = "Low"
        color = "🟢"
    elif proba < 0.8:
        risk_level = "Medium"
        color = "🟡"
    else:
        risk_level = "High"
        color = "🔴"


    # ----------------------------
    # Display Result
    # ----------------------------

    # Single-row output
    output = pd.DataFrame([{
        "Prediction (0=No,1=Yes)": int(pred),
        "Risk Level": f"{color} {risk_level}",
        "Probability (%)": round(proba * 100, 2)
    }])

    st.markdown("### 📊 Prediction Result")
    st.dataframe(output, width="stretch")
    st.write(f"**Predicted Probability:** {proba*100:.2f}%")
    st.write(f"**Risk Level:** {color} {risk_level}")

    st.info(
        """
**Probability Levels Explained:**
- Low (0 – 0.5) → Less chance of stroke
- Medium (0.5 – 0.8) → Moderate chance
- High (0.8 – 1.0) → Very high chance
"""
    )


# -------------------------------------------------
# Footer
# -------------------------------------------------
st.markdown("---")



Overwriting app.py


In [ ]:
import shutil

# Source path from Google Drive
source_path = '/content/drive/MyDrive/GNCIPL_AI_ML/best_stroke_model_augmented.pkl'

# Destination path (current directory where app.py resides)
destination_path = 'best_stroke_model_augmented.pkl'

try:
    shutil.copy(source_path, destination_path)
    print(f'Successfully copied {source_path} to {destination_path}')
except FileNotFoundError:
    print(f'Error: Source file not found at {source_path}. Please check the path and try again.')
except Exception as e:
    print(f'An error occurred while copying the file: {e}')

Successfully copied /content/drive/MyDrive/GNCIPL_AI_ML/best_stroke_model_augmented.pkl to best_stroke_model_augmented.pkl


In [ ]:
!ngrok authtoken 36krhmJyMIfpDYVN2vYhn4lhzOt_7ocQPfuTGyoV1z4sqSPcX

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

ngrok.kill()
public_url = ngrok.connect(addr='8501')
print(public_url)
!streamlit run --server.port 8501 app.py >/dev/null

NgrokTunnel: "https://inorganic-morgan-flagrant.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
import os
from shutil import copyfile

# Path to your existing app.py in Colab session
source_path = "app.py"  # if it's in the current working directory

# Destination folder in Google Drive
project_folder = "/content/drive/MyDrive/Stroke_GenAI_Project"
os.makedirs(project_folder, exist_ok=True)

# Copy app.py to Drive
destination_path = os.path.join(project_folder, "app.py")
copyfile(source_path, destination_path)

print("✅ app.py saved to Google Drive successfully!")
print(f"Location: {destination_path}")


✅ app.py saved to Google Drive successfully!
Location: /content/drive/MyDrive/Stroke_GenAI_Project/app.py
